In [2]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 3.0 MB/s eta 0:00:00a 0:00:01


In [3]:
import pandas as pd
import bs4
import requests
import openpyxl

## Web Scraping - BeautifulSoup
web scraping news from **thestandard**

In [4]:
i_page = 1
title_list = []
date_list = []
detail_list = []

while i_page <= 2:
  data = requests.get(f'https://thestandard.co/page/{i_page}/?s=%E0%B9%80%E0%B8%94%E0%B9%87%E0%B8%81%E0%B8%AB%E0%B8%A5%E0%B8%B8%E0%B8%94&search')
  soup = bs4.BeautifulSoup(data.text)
    
  for c in soup.find_all('div', class_='col-md-8'):
    title_list.append(c.find('h3', class_= 'news-title').find('a').text.strip())
    date_list.append(c.find('div', class_='date').text.strip())
    detail_list.append(c.find('div', class_='desc').text.strip())
  print('Complete page number: ',i_page)
  i_page += 1

Complete page number:  1
Complete page number:  2


In [123]:
data = {'title': title_list,
       'date': date_list ,
       'detail': detail_list}

table = pd.DataFrame(data)
#table = table.set_index('title')

## Prepare Data

In [124]:
table.columns
#check data type
table.dtypes
#check missing values
table.isna().sum().sort_values(ascending = False)

title     0
date      0
detail    0
dtype: int64

In [125]:
table['date_split'] = table['date'].str.split('\s')

In [126]:
table['D'] = table['date_split'].str[0]
table['M'] = table['date_split'].str[1]
table['Y']  = table['date_split'].str[2]

In [127]:
table['M'].unique()

array(['กันยายน', 'กุมภาพันธ์', 'สิงหาคม', 'กรกฎาคม', 'เมษายน', 'มกราคม',
       'ธันวาคม', 'พฤศจิกายน', 'มิถุนายน', 'พฤษภาคม', 'มีนาคม'],
      dtype=object)

In [128]:
table['M'].replace({
    'มกราคม':'01',
    'กุมภาพันธ์':'02',
    'มีนาคม' :'03',
    'เมษายน':'04',
    'พฤษภาคม':'05',
     'มิถุนายน':'06',
     'กรกฎาคม':'07',
    'สิงหาคม':'08',
    'กันยายน':'09',
    'ตุลาคม':'10',
    'พฤศจิกายน':'11',
    'ธันวาคม': '12'
    }, inplace=True)

In [129]:
table['DATE'] = table['D']+'-'+table['M']+'-'+table['Y']

In [130]:
table['DATE'] = pd.to_datetime(table['DATE'])

In [131]:
table.drop(['date', 'date_split', 'D','M','Y'], axis=1, inplace=True)

In [132]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   title   20 non-null     object        
 1   detail  20 non-null     object        
 2   DATE    20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 608.0+ bytes


In [133]:
table = table.to_dict('records')

In [134]:
table

[{'title': 'กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ. ป้องกันไม่ให้เด็กหลุดออกจากระบบ ตั้งเป้า 5 พันคน',
  'detail': 'วานนี้ (6 กันยายน) ที่ชุมชนโรงหวาย เขตสวนหลวง ศานนท์ หวังสร้างบุญ รองผู้ว่าราชการกรุงเทพมหานคร พร้อมด้วย ไกรยส ภัทราวาท ผู้จัดการกองทุนเพื่อความเสมอภาคทางการศึกษา (กสศ.) ปิยะวรรณ จระกา สมาชิกสภากรุงเทพมหานคร (ส.ก.) เขตสวนหลวง ผู้บริหารสำนักการศึกษา และผู้เกี่ยวข้องร่วมลงพื้นที่เพื่อค้นหาเด็กยากจนพิเศษในชุมชนที่มีความเสี่ยงหลุดจากระบบการศึกษา เพื่อล...',
  'DATE': Timestamp('2022-07-09 00:00:00')},
 {'title': 'เด็กทุกคนต้องได้เรียน! ‘แสนสิริ’ ออกหุ้นกู้ 100 ล้านบาท สนับสนุนกองทุนเพื่อความเสมอภาคทางการศึกษา นำร่องที่ จ.ราชบุรี ตั้งเป้าเด็กหลุดจากระบบการศึกษาเป็น ‘ศูนย์’ ภายใน 3 ปี',
  'detail': 'ในปี 2565 ‘แสนสิริ’ มีพันธกิจใหญ่ โดยมีเป้าหมายในการสนับสนุนกองทุนเพื่อความเสมอภาคทางการศึกษา หรือ กสศ. โดยมีความตั้งใจที่จะ ‘สร้างความเสมอภาค และลดความเหลื่อมล้ำด้านการศึกษา’ ด้วยการประกาศพันธกิจในโครงการ ‘Zero Dropout เด็กทุกคนต้องได้เรียน’ วางแผนพัฒนาคุณภาพการศึกษาไทยอย่า

In [119]:
with open('news.json', 'w', encoding ='utf8') as json_file:
    json.dump(table, json_file)